# Convert a PyTorch Model to OpenVINO™ IR

This tutorial demonstrates step-by-step instructions on how to do inference on a PyTorch classification model using OpenVINO Runtime.
Starting from OpenVINO 2023.0 release, OpenVINO supports direct PyTorch model conversion without an intermediate step to convert them into ONNX format. In order, if you try to use the lower OpenVINO version or prefer to use ONNX, please check this [tutorial](../pytorch-to-openvino/pytorch-onnx-to-openvino.ipynb).

In this tutorial, we will use the [RegNetY_800MF](https://arxiv.org/abs/2003.13678) model from [torchvision](https://pytorch.org/vision/stable/index.html) to demonstrate how to convert PyTorch models to OpenVINO Intermediate Representation.

The RegNet model was proposed in [Designing Network Design Spaces](https://arxiv.org/abs/2003.13678) by Ilija Radosavovic, Raj Prateek Kosaraju, Ross Girshick, Kaiming He, Piotr Dollár. The authors design search spaces to perform Neural Architecture Search (NAS). They first start from a high dimensional search space and iteratively reduce the search space by empirically applying constraints based on the best-performing models sampled by the current search space.  Instead of focusing on designing individual network instances, authors design network design spaces that parametrize populations of networks. The overall process is analogous to the classic manual design of networks but elevated to the design space level. The RegNet design space provides simple and fast networks that work well across a wide range of flop regimes.


#### Table of contents:

- [Prerequisites](#Prerequisites)
- [Load PyTorch Model](#Load-PyTorch-Model)
    - [Prepare Input Data](#Prepare-Input-Data)
    - [Run PyTorch Model Inference](#Run-PyTorch-Model-Inference)
    - [Benchmark PyTorch Model Inference](#Benchmark-PyTorch-Model-Inference)
- [Convert PyTorch Model to OpenVINO Intermediate Representation](#Convert-PyTorch-Model-to-OpenVINO-Intermediate-Representation)
    - [Select inference device](#Select-inference-device)
    - [Run OpenVINO Model Inference](#Run-OpenVINO-Model-Inference)
    - [Benchmark OpenVINO Model Inference](#Benchmark-OpenVINO-Model-Inference)
- [Convert PyTorch Model with Static Input Shape](#Convert-PyTorch-Model-with-Static-Input-Shape)
    - [Select inference device](#Select-inference-device)
    - [Run OpenVINO Model Inference with Static Input Shape](#Run-OpenVINO-Model-Inference-with-Static-Input-Shape)
    - [Benchmark OpenVINO Model Inference with Static Input Shape](#Benchmark-OpenVINO-Model-Inference-with-Static-Input-Shape)

### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend  running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/pytorch-to-openvino/pytorch-to-openvino.ipynb" />


## Prerequisites
[back to top ⬆️](#Table-of-contents:)


Install notebook dependencies

In [1]:
import sys
!{sys.executable} -m pip install openvino>=2023.1.0 scipy Pillow torch torchvision --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
import os
import sys
username = os.environ.get('USER')
user_bin_path = os.path.expanduser(f"/home/{username}/.local/bin")
sys.path.append(user_bin_path)
print(sys.path)

Please restart the kernel in order to make installed dependencies available

Download input data and label map

In [2]:
import requests
from pathlib import Path
from PIL import Image

MODEL_DIR = Path("model")
DATA_DIR = Path("data")

MODEL_DIR.mkdir(exist_ok=True)
DATA_DIR.mkdir(exist_ok=True)
MODEL_NAME = "regnet_y_800mf"

image = Image.open(requests.get("https://farm9.staticflickr.com/8225/8511402100_fea15da1c5_z.jpg", stream=True).raw)

labels_file = DATA_DIR / "imagenet_2012.txt"

if not labels_file.exists():
    resp = requests.get("https://raw.githubusercontent.com/openvinotoolkit/open_model_zoo/master/data/dataset_classes/imagenet_2012.txt")
    with labels_file.open("wb") as f:
        f.write(resp.content)

imagenet_classes = labels_file.open("r").read().splitlines()

## Load PyTorch Model
[back to top ⬆️](#Table-of-contents:)

Generally, PyTorch models represent an instance of the `torch.nn.Module` class, initialized by a state dictionary with model weights.
Typical steps for getting a pre-trained model:

1. Create an instance of a model class
2. Load checkpoint state dict, which contains pre-trained model weights
3. Turn the model to evaluation for switching some operations to inference mode

The `torchvision` module provides a ready-to-use set of functions for model class initialization. We will use `torchvision.models.regnet_y_800mf`. You can directly pass pre-trained model weights to the model initialization function using the weights enum `RegNet_Y_800MF_Weights.DEFAULT`.

In [3]:
import torchvision

# get default weights using available weights Enum for model
weights = torchvision.models.RegNet_Y_800MF_Weights.DEFAULT

# create model topology and load weights
model = torchvision.models.regnet_y_800mf(weights=weights)

# switch model to inference mode
model.eval();

### Prepare Input Data
[back to top ⬆️](#Table-of-contents:)

The code below demonstrates how to preprocess input data using a model-specific transforms module from `torchvision`. After transformation, we should concatenate images into batched tensor, in our case, we will run the model with batch 1, so we just unsqueeze input on the first dimension.

In [4]:
import torch

# Initialize the Weight Transforms
preprocess = weights.transforms()

# Apply it to the input image
img_transformed = preprocess(image)

# Add batch dimension to image tensor
input_tensor = img_transformed.unsqueeze(0)

### Run PyTorch Model Inference
[back to top ⬆️](#Table-of-contents:)

The model returns a vector of probabilities in raw logits format, softmax can be applied to get normalized values in the [0, 1] range. For a demonstration that the output of the original model and OpenVINO converted is the same, we defined a common postprocessing function which can be reused later.

In [ ]:
import numpy as np
from scipy.special import softmax

# Perform model inference on input tensor
result = model(input_tensor)


# Postprocessing function for getting results in the same way for both PyTorch model inference and OpenVINO
def postprocess_result(output_tensor: np.ndarray, top_k: int = 5):
    """
    Posprocess model results. This function applied sofrmax on output tensor and returns specified top_k number of labels with highest probability
    Parameters:
      output_tensor (np.ndarray): model output tensor with probabilities
      top_k (int, *optional*, default 5): number of labels with highest probability for return
    Returns:
      topk_labels: label ids for selected top_k scores
      topk_scores: selected top_k highest scores predicted by model
    """
    softmaxed_scores = softmax(output_tensor, -1)[0]
    topk_labels = np.argsort(softmaxed_scores)[-top_k:][::-1]
    topk_scores = softmaxed_scores[topk_labels]
    return topk_labels, topk_scores


# Postprocess results
top_labels, top_scores = postprocess_result(result.detach().numpy())

# Show results
display(image)
for idx, (label, score) in enumerate(zip(top_labels, top_scores)):
    _, predicted_label = imagenet_classes[label].split(" ", 1)
    print(f"{idx + 1}: {predicted_label} - {score * 100 :.2f}%")

### Benchmark PyTorch Model Inference
[back to top ⬆️](#Table-of-contents:)


In [ ]:
%%timeit

# Run model inference
model(input_tensor)

## Convert PyTorch Model to OpenVINO Intermediate Representation
[back to top ⬆️](#Table-of-contents:)

Starting from the 2023.0 release OpenVINO supports direct PyTorch models conversion to OpenVINO Intermediate Representation (IR) format. OpenVINO model conversion API should be used for these purposes. More details regarding PyTorch model conversion can be found in OpenVINO [documentation](https://docs.openvino.ai/2024/openvino-workflow/model-preparation/convert-model-pytorch.html)


The `convert_model` function accepts the PyTorch model object and returns the `openvino.Model` instance ready to load on a device using `core.compile_model` or save on disk for next usage using `ov.save_model`. Optionally, we can provide additional parameters, such as:

* `compress_to_fp16` - flag to perform model weights compression into FP16 data format. It may reduce the required space for model storage on disk and give speedup for inference devices, where FP16 calculation is supported.
* `example_input` - input data sample which can be used for model tracing.
* `input_shape` - the shape of input tensor for conversion

and any other advanced options supported by model conversion Python API. More details can be found on this [page](https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html)

In [ ]:
import openvino as ov

# Create OpenVINO Core object instance
core = ov.Core()

# Convert model to openvino.runtime.Model object
ov_model = ov.convert_model(model)

# Save openvino.runtime.Model object on disk
ov.save_model(ov_model, MODEL_DIR / f"{MODEL_NAME}_dynamic.xml")

ov_model

### Select inference device
[back to top ⬆️](#Table-of-contents:)

select device from dropdown list for running inference using OpenVINO

In [ ]:
import ipywidgets as widgets

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="AUTO",
    description="Device:",
    disabled=False,
)

device

In [ ]:
# Load OpenVINO model on device
compiled_model = core.compile_model(ov_model, device.value)
compiled_model

### Run OpenVINO Model Inference
[back to top ⬆️](#Table-of-contents:)


In [ ]:
# Run model inference
result = compiled_model(input_tensor)[0]

# Posptorcess results
top_labels, top_scores = postprocess_result(result)

# Show results
display(image)
for idx, (label, score) in enumerate(zip(top_labels, top_scores)):
    _, predicted_label = imagenet_classes[label].split(" ", 1)
    print(f"{idx + 1}: {predicted_label} - {score * 100 :.2f}%")

### Benchmark OpenVINO Model Inference
[back to top ⬆️](#Table-of-contents:)


In [ ]:
%%timeit

compiled_model(input_tensor)

## Convert PyTorch Model with Static Input Shape
[back to top ⬆️](#Table-of-contents:)


The default conversion path preserves dynamic input shapes, in order if you want to convert the model with static shapes, you can explicitly specify it during conversion using the `input_shape` parameter or reshape the model into the desired shape after conversion. For the model reshaping example please check the following [tutorial](../openvino-api/openvino-api.ipynb).

In [ ]:
# Convert model to openvino.runtime.Model object
ov_model = ov.convert_model(model, input=[[1, 3, 224, 224]])
# Save openvino.runtime.Model object on disk
ov.save_model(ov_model, MODEL_DIR / f"{MODEL_NAME}_static.xml")
ov_model

### Select inference device
[back to top ⬆️](#Table-of-contents:)

select device from dropdown list for running inference using OpenVINO

In [13]:
device

Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

In [ ]:
# Load OpenVINO model on device
compiled_model = core.compile_model(ov_model, device.value)
compiled_model

Now, we can see that input of our converted model is tensor of shape [1, 3, 224, 224] instead of [?, 3, ?, ?] reported by previously converted model.

### Run OpenVINO Model Inference with Static Input Shape
[back to top ⬆️](#Table-of-contents:)


In [ ]:
# Run model inference
result = compiled_model(input_tensor)[0]

# Posptorcess results
top_labels, top_scores = postprocess_result(result)

# Show results
display(image)
for idx, (label, score) in enumerate(zip(top_labels, top_scores)):
    _, predicted_label = imagenet_classes[label].split(" ", 1)
    print(f"{idx + 1}: {predicted_label} - {score * 100 :.2f}%")

### Benchmark OpenVINO Model Inference with Static Input Shape
[back to top ⬆️](#Table-of-contents:)


In [ ]:
%%timeit

compiled_model(input_tensor)